# LangChain(v0.3) 의 주요 Components
https://python.langchain.com/docs/how_to/#components

# prompts
https://python.langchain.com/api_reference/core/prompts.html#module-langchain_core.prompts

In [ ]:
"""
prompts class 계층도

BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate
"""
None

# FewShotPromptTemplate
- 모델에 예제(example) 주기 

In [1]:
# v0.3
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

In [4]:
# 직전에 배운 PromptTemplate
# 방법1
t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country='France')

'What is the capital of France'

In [ ]:
# 템플릿을 만드는 이점 또 한가지!
# prompt template 을 디스크에 '저장'하고 'load' 할 수 있기 떼문이다.

# 나중에 LLM 다룰때 prompt 는 매우 중요합니다.
# 대규모 프로젝트에서는 
# - 'prompt 만 만드는 팀'이 있고
# - '코딩하는 팀'이 따로 있을것이다.  
#    데이터베이스나 파일 등에 만들어 저장해 놓은 prompt 를 load 해야 할겁니다

In [5]:
# 방법2
t2 = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=['country']  # 입력변수들을 PromptTemplate 에 알려준다
)

t2.format(country='France')

'What is the capital of France'

In [ ]:
# 모델에게 '어떻게 대답해야 하는 지에 대한 예제(example)'를 AI 모델에게 주는 것이
# prompt 를 사용해서 '어떻게 대답해야 하는지 알려주는 것'보다 훨씬 좋다

# FewShotPromptTemplate 이 하는 일이 바로 그거다!
# - 이를 통해 예제(샘플)를 형식화(포맷) 할수 있다.
# - 이런 예제들을 데이터베이스등에 저장시켜놓고 활용할수도 있다


In [6]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate

In [11]:
# ↓ 이 예제들은
# 모델이 나에게 '이런 식으로 답변해 줬으면 좋겠다' 하는 예제들입니다

examples = [
  {
    "question": "What do you know about France?",

    # ↓ 원하는 형식의 답변 이다..
    "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      """,
  },
  {
    "question": "What do you know about Italy?",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },
]


In [12]:
# 만약 '예제(example) ' 없이 전달하면?
chat.invoke("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower and the Louvre Museum. France is also known for its wine production, fashion industry, and art scene. The country has a diverse landscape, including mountains, beaches, and countryside. French is the official language, and the currency is the Euro. France is a member of the European Union and is one of the most visited countries in the world.

AIMessage(content='France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower and the Louvre Museum. France is also known for its wine production, fashion industry, and art scene. The country has a diverse landscape, including mountains, beaches, and countryside. French is the official language, and the currency is the Euro. France is a member of the European Union and is one of the most visited countries in the world.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--688b886e-d68b-4200-b147-e2d53828fcad-0')

In [8]:
# FewShotPromptTemplate 을 사용하여 만들기.
example_template = """
    Human: {question}
    AI: {answer}
"""
# ↑ {question} 과 {answer} 는 위 샘플과 동일한 key를 사용하여 작성한다.

In [9]:
example_prompt = PromptTemplate.from_template(example_template)

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question}\n    AI: {answer}\n')

In [13]:
prompt = FewShotPromptTemplate(
    example_prompt= example_prompt,   # 사용할 prompt
    examples = examples, 

    # ↑ 이렇게만 해도 랭체인이 알아서 각각의 예제 리스트들을
    #  위 prompt 를 사용하여 포맷팅(형식화) 합니다

    # 여기에 유저의 질문을 넣어줄겁니다

    # suffix=
    # 형식화된 모든 예제 마지막에 나오는 내용.
    # 그래서 한번 다 형식화가 끝난 다음, 사용자의 질문은 무엇인지 나옵니다

    # 여기서는 위에 있는 동일한 형식을 사용할 겁니다  

    suffix="Human: What do you know about {country}?",

    #  어떤 변수를 suffic 에서 사용할 것인지 지정해 주어야 한다
    input_variables=['country']
)

prompt

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n      Here is what I know:\n      Capital: Paris\n      Language: French\n      Food: Wine and Cheese\n      Currency: Euro\n      '}, {'question': 'What do you know about Italy?', 'answer': '\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      '}, {'question': 'What do you know about Greece?', 'answer': '\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      Currency: Euro\n      '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question}\n    AI: {answer}\n'), suffix='Human: What do you know about {country}?')

In [14]:
print(prompt.format(country='Germany'))


    Human: What do you know about France?
    AI: 
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      



    Human: What do you know about Italy?
    AI: 
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      



    Human: What do you know about Greece?
    AI: 
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      


Human: What do you know about Germany?


In [ ]:
# step1  example 리스트를 만들고   examples
# step2  FewShotPromptTemplate 에 전달했고 examples=
# step3  어떻게 전달한 예제들을 형식화 할지 알려주었고
# step4  마지막에 질문을 포함시켰다.  suffix, input_variables

# AI 는 우리의 예제들과 똑같은 구조, 형태로 답변하게 될겁니다

In [15]:
chain = prompt | chat

In [16]:
chain.invoke({
    "country": "Germany",
})

AI: 
      Here is what I know:
      Capital: Berlin
      Language: German
      Food: Bratwurst and Sauerkraut
      Currency: Euro

AIMessage(content='AI: \n      Here is what I know:\n      Capital: Berlin\n      Language: German\n      Food: Bratwurst and Sauerkraut\n      Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--66d85fdc-88ad-468c-8b8e-4ef5fbb23d24-0')

In [17]:
chain.invoke({
    "country": "Turkey",
})

AI: 
      Here is what I know:
      Capital: Ankara
      Language: Turkish
      Food: Kebab and Baklava
      Currency: Turkish Lira

AIMessage(content='AI: \n      Here is what I know:\n      Capital: Ankara\n      Language: Turkish\n      Food: Kebab and Baklava\n      Currency: Turkish Lira', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--4a8a6dda-8481-419f-960c-262e14c3b5cf-0')

# FewShotChatMessagePromptTemplate

In [18]:
# v0.3
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
# https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate.html

# v0.3
from langchain_core.prompts.chat import ChatPromptTemplate
# https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

In [19]:
examples = [
  {
    "country": "France",  # <-- 변경
    "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      """,
  },
  {
    "country": "Italy",  # <-- 변경
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "country": "Greece",  # <-- 변경
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },
]


In [20]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt, 
    examples=examples,
    # suffix= 등은 필요없다.
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,  # !!
    ("human", "What do you know about {country}?")
])



In [21]:
chain = final_prompt | chat

In [22]:
chain.invoke({"country": "Germany"})


      I know this:
      Capital: Berlin
      Language: German
      Food: Bratwurst and Sauerkraut
      Currency: Euro
      

AIMessage(content='\n      I know this:\n      Capital: Berlin\n      Language: German\n      Food: Bratwurst and Sauerkraut\n      Currency: Euro\n      ', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--d5217692-cf4d-4bbc-b206-d31da08fb53f-0')

In [23]:
#  때로는 수천개의 예제를 가지고 있을텐데,  이를 모두 모델에게 줄수 없는 상황이 있을수 있다.
#    이유1) 비용이 많이 든다..  많은 텍스트 땜에.
#    이유2) '허용하는 범위' 라는게 있다 => 모~든 예제들을 모델에게 줄 수는 없다.  제한이 있다 (context window)

#  그래서 예제를 선별하는 방법에 대해 배워보자

# LengthBasedExampleSelector

In [24]:
# v0.3
from langchain_core.example_selectors.length_based import LengthBasedExampleSelector
# https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.length_based.LengthBasedExampleSelector.html#langchain_core.example_selectors.length_based.LengthBasedExampleSelector

In [25]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [26]:
example_prompt = PromptTemplate.from_template("Human: {country}\nAI: {answer}")

example_prompt

PromptTemplate(input_variables=['answer', 'country'], input_types={}, partial_variables={}, template='Human: {country}\nAI: {answer}')

In [27]:
# example selector 생성
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt, 
    max_length=10,    # 예제의 양을 얼마나 허용할 지 정해주기. max_length 값 을 넘어가는 예제는 cut out 된다
)

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt, 
    # examples = examples, 
    example_selector=example_selector,  # <- max_length= 값에 따라 예제의 양을 정해줄 수 있다.
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)


In [28]:
prompt.format(country='Brazil')

'Human: What do you know about Brazil?'

In [ ]:
"""
'Human: What do you know about Brazil?'

선택된 example 이 없다?  => max_length= 값을 변경해보자
"""
None

In [30]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt, 
    max_length=80,   # <-- 값을 변경!
)

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt, 
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country='Brazil'))

Human: France
AI: 
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        

Human: What do you know about Brazil?


In [31]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt, 
    max_length=180,   # <-- 값을 좀더 크게 하면?
)

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt, 
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country='Brazil'))

Human: France
AI: 
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        

Human: Italy
AI: 
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Brazil?


## Custom ExampleSelector (BaseExampleSelector)

In [ ]:
# 램덤하게 exmaple 를 select 하는 ExampleSelector 를 만들어 보자.

In [32]:
# v0.3
from langchain_core.example_selectors.base import BaseExampleSelector

In [35]:
# BaseExampleSelector 의 구현객체를 만드려면
#  상속 받은뒤 select_examples() 과 add_example() 을 반드시 오버라이딩 해주어야 한다.

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    # select_examples()
    # 입력에 따라 어떠한 샘플을 사용할지 select 함.
    
    # 이번 예제에서는 examples 리스트 에서 random 으로 선택하게 하려 함.
    # ※ 이는 얼마든지 복잡하게 만들어 볼수도 있다.
    def select_examples(self, input_variables):    
        from random import choice
        return [choice(self.examples)]   # 무작위로 선택 1개 한 뒤, list에 담아 리턴.

    # add_example()
    # Add new example to store.  이미 존재하는 example 에 example 을 추가하는 method
    def add_example(self, example):
        self.examples.append(example)
        

In [89]:
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt, 
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country='Brazil'))

Human: Greece
AI: 
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        

Human: What do you know about Brazil?


# PromptTemplate 저장 / 읽기

## json 파일 

In [90]:
# prompt.json 파일을 생성합니다
with open('prompt.json', 'w') as f:
    f.write("""
    {
      "_type":"prompt",
      "template":"What is the capital of {country}",
      "input_variables":["country"]
    }
    """)

In [91]:
# v0.3
from langchain_core.prompts.loading import load_prompt
# https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.loading.load_prompt.html
# Unified method for loading a prompt from LangChainHub or local fs.

In [92]:
prompt = load_prompt('./prompt.json')

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}')

In [93]:
# 포맷팅
prompt.format(country='Germany')

'What is the capital of Germany'

## yaml 파일 

In [96]:
with open('prompt.yaml', 'w') as f:
    f.write("""
_type: "prompt"
template: "What is the capital of {country}"
input_variables: ["country"]
    """)

In [97]:
prompt = load_prompt("./prompt.yaml")
print(prompt)

prompt.format(country='China')

input_variables=['country'] input_types={} partial_variables={} template='What is the capital of {country}'


'What is the capital of China'

# Caching

## set_llm_cache(),  InMemoryCache

In [98]:
# v0.3
from langchain_core.globals import set_llm_cache
# https://python.langchain.com/api_reference/core/globals/langchain_core.globals.set_llm_cache.html#set-llm-cache
# Set a new LLM cache, overwriting the previous value, if any.

# v0.3
from langchain_core.caches import InMemoryCache
# https://python.langchain.com/api_reference/core/caches/langchain_core.caches.InMemoryCache.html

# Cache that stores things in 'memory'.
# Initialize with empty cache.

In [99]:
set_llm_cache(InMemoryCache())  # <- LLM 의 모든 response 가 '메모리' 에 저장(cache) 된다

In [100]:
chat = ChatOpenAI(
    temperature=0.1,
)

In [101]:
# 동일한 질문을 두번 해볼거다.  
# 우선 시간 측정하는 함수도 준비해보자.

import time
from datetime import timedelta

def check_laptime(message):
    start_time = time.time()
    response = chat.invoke(message)
    end_time = time.time()
    elapsed_time = end_time - start_time # 경과 시간.
    print('▶ 경과시간 %s' % (str(timedelta(seconds = elapsed_time))))
    print(f'{len(response.content)} 글자: {response.content}\n')




In [102]:
check_laptime("How do you make Italian pasta?")

▶ 경과시간 0:00:04.792942
966 글자: To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- Pinch of salt

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and create a well in the center.
2. Crack the eggs into the well and add a pinch of salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it becomes smooth and elastic.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.
7. Cut the rolled-out dough into desired shapes such as fettuccine, spaghetti, or ravioli.
8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.
9. Drain the pasta and toss with your favorite sauce or toppings.

Enjoy your homemade Italian pasta!



In [103]:
check_laptime("How do you make Italian pasta?")

▶ 경과시간 0:00:00
966 글자: To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- Pinch of salt

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and create a well in the center.
2. Crack the eggs into the well and add a pinch of salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it becomes smooth and elastic.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.
7. Cut the rolled-out dough into desired shapes such as fettuccine, spaghetti, or ravioli.
8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.
9. Drain the pasta and toss with your favorite sauce or toppings.

Enjoy your homemade Italian pasta!



## set_debug()

In [104]:
# v0.3
from langchain_core.globals import set_debug
# https://python.langchain.com/api_reference/core/globals/langchain_core.globals.set_debug.html#langchain_core.globals.set_debug
# Set a new value for the debug global setting.

In [105]:
set_debug(True)

In [106]:
chat.invoke("How do you make Italian pasta?")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Italian pasta?"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.

AIMessage(content='To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.\n7. Cut the rolled-out dough into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pas

In [109]:
set_debug(False)

## SQLiteCache

In [107]:
# v0.3
from langchain_community.cache import SQLiteCache
# Cache that uses SQLite as a backend.
# Initialize by creating the engine and all tables.
# https://python.langchain.com/api_reference/community/cache/langchain_community.cache.SQLiteCache.html

In [108]:
# 데이터베이스에 cache
set_llm_cache(SQLiteCache('cache.db'))

In [110]:
chat.invoke("How do you make Italian pasta?")

AIMessage(content='To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!', additional_kwargs={'refusal': None}, response_metadata={'

In [111]:
chat.invoke("How do you make Italian pasta?")

AIMessage(content='To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!', additional_kwargs={'refusal': None}, response_metadata={'

## debug, cache 끄기

In [112]:
set_debug(False)
set_llm_cache(None)

# Serialization
- 모델 저장, 불러오기

## OpenAI 모델 비용 확인 

In [1]:
# v0.3
from langchain_community.callbacks.manager import get_openai_callback
# Get the OpenAI callback handler in a context manager.
# which conveniently exposes token and cost information.
# Returns: The OpenAI callback handler.
# https://python.langchain.com/api_reference/community/callbacks/langchain_community.callbacks.manager.get_openai_callback.html#get-openai-callback

In [2]:
from langchain_openai.chat_models.base import ChatOpenAI

In [3]:
chat = ChatOpenAI(
    temperature=0.1,
)

In [4]:
with get_openai_callback() as usage:
    # 이 안에서 모델 호출.
    chat.invoke("What is the recipe for soju")
    print(usage)

Tokens Used: 220
	Prompt Tokens: 14
		Prompt Tokens Cached: 0
	Completion Tokens: 206
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.000316


In [5]:
with get_openai_callback() as usage:
    # 이 안에서 모델 호출.
    a = chat.invoke("What is the recipe for soju")
    b = chat.invoke("What is the recipe for bread")
    print('💚\n', a.content)
    print('💛\n', b.content)
    print(usage)

💚
 Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of yeast

Instructions:
1. Rinse the rice thoroughly and soak it in water for at least 1 hour.
2. Drain the rice and steam it until fully cooked.
3. Let the rice cool down to room temperature.
4. In a large bowl, mix the nuruk and yeast with water until dissolved.
5. Add the cooked rice to the bowl and mix well.
6. Cover the bowl with a clean cloth and let it ferment in a warm place for 3-4 days.
7. After fermentation, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean bottle and store it in a cool, dark place for at least 1 week before serving.

Enjoy your homemade soju!
💛
 Ingredients:
- 4 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 2 tablespoons sugar
- 2 teaspoons salt
- 2 tablespoons olive oil

Instructions:
1. In a large mixing bowl, combine the warm water, sugar, and yeast. Let it sit for abo

In [6]:
print(usage.total_cost)

0.0007050000000000001


In [7]:
print(usage.total_tokens)

488


## 모델 config 를 저장하고 불러오기

In [8]:
# LLM 모델
from langchain_openai.llms.base import OpenAI

In [9]:
llm = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model='gpt-4-turbo'
)

# 모델 config 를 저장
llm.save('model.json')

In [10]:
!type model.json

{
    "model_name": "gpt-4-turbo",
    "temperature": 0.1,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "n": 1,
    "seed": null,
    "logprobs": null,
    "max_tokens": 450,
    "_type": "openai"
}


In [11]:
# 저장된 모델 (config) 를 다시 불러오기
# v0.3
from langchain_community.llms.loading import load_llm
# Load LLM from a file.
# https://python.langchain.com/api_reference/community/llms/langchain_community.llms.loading.load_llm.html#langchain_community.llms.loading.load_llm

In [13]:
llm = load_llm('model.json')

D:\NLP2501\NLPWork\.venv\Lib\site-packages\langchain_community\llms\openai.py:255: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
D:\NLP2501\NLPWork\.venv\Lib\site-packages\langchain_community\llms\openai.py:1089: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [14]:
llm

OpenAIChat(client=APIRemovedInV1Proxy, model_kwargs={'model_name': 'gpt-4-turbo', 'temperature': 0.1, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'seed': None, 'logprobs': None, 'max_tokens': 450}, prefix_messages=[])